# T1558 Steal or Forge Kerberos Tickets

-----------------------------------------------------------------------

## Technique Description

Adversaries may attempt to subvert Kerberos authentication by stealing or forging Kerberos tickets to enable [Pass the Ticket](https://attack.mitre.org/techniques/T1550/003). Kerberos is an authentication protocol widely used in modern Windows domain environments. In Kerberos environments, referred to as “realms”, there are three basic participants: client, service, and Key Distribution Center (KDC).(Citation: ADSecurity Kerberos Ring Decoder) Clients request access to a service and through the exchange of Kerberos tickets, originating from KDC, they are granted access after having successfully authenticated. The KDC is responsible for both authentication and ticket granting.  Adversaries may attempt to abuse Kerberos by stealing tickets or forging tickets to enable unauthorized access.

On Windows, the built-in <code>klist</code> utility can be used to list and analyze cached Kerberos tickets.(Citation: Microsoft Klist)

Linux systems on Active Directory domains store Kerberos credentials locally in the credential cache file referred to as the "ccache". The credentials are stored in the ccache file while they remain valid and generally while a user's session lasts.(Citation: MIT ccache) On modern Redhat Enterprise Linux systems, and derivative distributions, the System Security Services Daemon (SSSD) handles Kerberos tickets. By default SSSD maintains a copy of the ticket database that can be found in <code>/var/lib/sss/secrets/secrets.ldb</code> as well as the corresponding key located in <code>/var/lib/sss/secrets/.secrets.mkey</code>. Both files require root access to read. If an adversary is able to access the database and key, the credential cache Kerberos blob can be extracted and converted into a usable Kerberos ccache file that adversaries may use for [Pass the Ticket](https://attack.mitre.org/techniques/T1550/003). The ccache file may also be converted into a Windows format using tools such as Kekeo.(Citation: Linux Kerberos Tickets)(Citation: Brining MimiKatz to Unix)(Citation: Kekeo)


Kerberos tickets on macOS are stored in a standard ccache format, similar to Linux. By default, access to these ccache entries is federated through the KCM daemon process via the Mach RPC protocol, which uses the caller's environment to determine access. The storage location for these ccache entries is influenced by the <code>/etc/krb5.conf</code> configuration file and the <code>KRB5CCNAME</code> environment variable which can specify to save them to disk or keep them protected via the KCM daemon. Users can interact with ticket storage using <code>kinit</code>, <code>klist</code>, <code>ktutil</code>, and <code>kcc</code> built-in binaries or via Apple's native Kerberos framework. Adversaries can use open source tools to interact with the ccache files directly or to use the Kerberos framework to call lower-level APIs for extracting the user's TGT or Service Tickets.(Citation: SpectorOps Bifrost Kerberos macOS 2019)(Citation: macOS kerberos framework MIT)


## Technique Detection

Monitor for anomalous Kerberos activity, such as malformed or blank fields in Windows logon/logoff events (Event ID 4624, 4672, 4634), RC4 encryption within ticket granting tickets (TGTs), and ticket granting service (TGS) requests without preceding TGT requests.(Citation: ADSecurity Detecting Forged Tickets)(Citation: Stealthbits Detect PtT 2019)(Citation: CERT-EU Golden Ticket Protection)

Monitor the lifetime of TGT tickets for values that differ from the default domain duration.(Citation: Microsoft Kerberos Golden Ticket)

Monitor for indications of [Pass the Ticket](https://attack.mitre.org/techniques/T1550/003) being used to move laterally. 

Enable Audit Kerberos Service Ticket Operations to log Kerberos TGS service ticket requests. Particularly investigate irregular patterns of activity (ex: accounts making numerous requests, Event ID 4769, within a small time frame, especially if they also request RC4 encryption [Type 0x17]).(Citation: Microsoft Detecting Kerberoasting Feb 2018) (Citation: AdSecurity Cracking Kerberos Dec 2015)

Monitor for unexpected processes interacting with lsass.exe.(Citation: Medium Detecting Attempts to Steal Passwords from Memory) Common credential dumpers such as [Mimikatz](https://attack.mitre.org/software/S0002) access the LSA Subsystem Service (LSASS) process by opening the process, locating the LSA secrets key, and decrypting the sections in memory where credential details, including Kerberos tickets, are stored.

Monitor for unusual processes accessing <code>secrets.ldb</code> and <code>.secrets.mkey</code> located in <code>/var/lib/sss/secrets/</code>.

-----------------------------------------------------------------------

### Tactics:

  *   Credential-Access

### Platforms:

  * Windows

  * Linux

  * macOS

### Adversary Required Permissions:

  * User

  * root

### System Requirements:

  * Kerberos authentication enabled

### Data Sources:

  * **Active Directory:** Active Directory Credential Request

  * **Logon Session:** Logon Session Metadata

  * **File:** File Access

  * **Command:** Command Execution

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1558)

  * [Capec](https://capec.mitre.org/data/definitions/652.html)

  * [Adsecurity Kerberos Ring Decoder](https://adsecurity.org/?p=227), Sean Metcalf. (2014, September 12). Kerberos, Active Directory’s Secret Decoder Ring. Retrieved February 27, 2020.

  * [Microsoft Klist](https://docs.microsoft.com/windows-server/administration/windows-commands/klist), Microsoft. (2021, March 3). klist. Retrieved October 14, 2021.

  * [Mit Ccache](https://web.mit.edu/kerberos/krb5-1.12/doc/basic/ccache_def.html), Massachusetts Institute of Technology. (n.d.). MIT Kerberos Documentation: Credential Cache. Retrieved October 4, 2021.

  * [Linux Kerberos Tickets](https://www.fireeye.com/blog/threat-research/2020/04/kerberos-tickets-on-linux-red-teams.html), Trevor Haskell. (2020, April 1). Kerberos Tickets on Linux Red Teams. Retrieved October 4, 2021.

  * [Brining Mimikatz To Unix](https://labs.portcullis.co.uk/download/eu-18-Wadhwa-Brown-Where-2-worlds-collide-Bringing-Mimikatz-et-al-to-UNIX.pdf), Tim Wadhwa-Brown. (2018, November). Where 2 worlds collide Bringing Mimikatz et al to UNIX. Retrieved October 13, 2021.

  * [Kekeo](https://github.com/gentilkiwi/kekeo), Benjamin Delpy. (n.d.). Kekeo. Retrieved October 4, 2021.

  * [Spectorops Bifrost Kerberos Macos 2019](https://posts.specterops.io/when-kirbi-walks-the-bifrost-4c727807744f), Cody Thomas. (2019, November 14). When Kirbi walks the Bifrost. Retrieved October 6, 2021.

  * [Macos Kerberos Framework Mit](http://web.mit.edu/macdev/KfM/Common/Documentation/preferences.html), Massachusetts Institute of Technology. (2007, October 27). Kerberos for Macintosh Preferences Documentation. Retrieved October 6, 2021.

  * [Adsecurity Detecting Forged Tickets](https://adsecurity.org/?p=1515), Metcalf, S. (2015, May 03). Detecting Forged Kerberos Ticket (Golden Ticket & Silver Ticket) Use in Active Directory. Retrieved December 23, 2015.

  * [Stealthbits Detect Ptt 2019](https://blog.stealthbits.com/detect-pass-the-ticket-attacks), Jeff Warren. (2019, February 19). How to Detect Pass-the-Ticket Attacks. Retrieved February 27, 2020.

  * [Cert-Eu Golden Ticket Protection](https://cert.europa.eu/static/WhitePapers/UPDATED%20-%20CERT-EU_Security_Whitepaper_2014-007_Kerberos_Golden_Ticket_Protection_v1_4.pdf), Abolins, D., Boldea, C., Socha, K., Soria-Machado, M. (2016, April 26). Kerberos Golden Ticket Protection. Retrieved July 13, 2017.

  * [Microsoft Kerberos Golden Ticket](https://gallery.technet.microsoft.com/scriptcenter/Kerberos-Golden-Ticket-b4814285), Microsoft. (2015, March 24). Kerberos Golden Ticket Check (Updated). Retrieved February 27, 2020.

  * [Microsoft Detecting Kerberoasting Feb 2018](https://blogs.technet.microsoft.com/motiba/2018/02/23/detecting-kerberoasting-activity-using-azure-security-center/), Bani, M. (2018, February 23). Detecting Kerberoasting activity using Azure Security Center. Retrieved March 23, 2018.

  * [Adsecurity Cracking Kerberos Dec 2015](https://adsecurity.org/?p=2293), Metcalf, S. (2015, December 31). Cracking Kerberos TGS Tickets Using Kerberoast – Exploiting Kerberos to Compromise the Active Directory Domain. Retrieved March 22, 2018.

  * [Medium Detecting Attempts To Steal Passwords From Memory](https://medium.com/threatpunter/detecting-attempts-to-steal-passwords-from-memory-558f16dce4ea), French, D. (2018, October 2). Detecting Attempts to Steal Passwords from Memory. Retrieved October 11, 2019.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 8 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Information Here

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| Example APT/Threat actor | 1, 2 | 2, 3, 5|

## Detection Blindspots

- The most critical event for the analytic is 4769 which has to be enabled manually, and produces a significant amount of logs. Without performance tuning most organizations turn this auditing off due to volume. 

## Analytical References

  * https://posts.specterops.io/capability-abstraction-fbeaeeb26384
* https://posts.specterops.io/kerberosity-killed-the-domain-an-offensive-kerberos-overview-eb04b1402c61
* https://www.redsiege.com/blog/2020/10/detecting-kerberoasting/
* https://www.pentestpartners.com/security-blog/honeyroasting-how-to-detect-kerberoast-breaches-with-honeypots/
* https://www.trustedsec.com/blog/art_of_kerberoast/
* https://stealthbits.com/blog/how-to-detect-pass-the-ticket-attacks/
* https://adsecurity.org/?p=1515
* https://adsecurity.org/?p=3458

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** 'Windows API Function'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Kibana'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```InitializeSecurityContext OR LsaCallAuthenticationPackage```

#### Analytic 2

  * **Information:** 'Windows RPC'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Kibana'

  * **Notes:** 'Used by majority of Kerberoasting tooling except for Reubus'

  * **Query:** ```4f32adc8–6052–4a04–8701–293ccf2096f0 sspisrv.dll```

#### Analytic 3

  * **Information:** 'Identify "user-to-user krb_tgt_reply"'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Kibana, sysmon'

  * **Notes:** 'Used by majority of Kerberoasting tooling except for Reubus'

  * **Query:** ```event_id: 4769 AND event_data.Status: "0x0" AND event_data.TicketEncryptionType: "0x17" AND not service_name: "krbtgt" AND not event_data.ServiceName: "*$" AND not event_data.TargetUserName: "*$@*"```

-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** 'Kerberos TGS-REQ/REP packets'

  * **Source:** 'Network Logs'

  * **Tool:** 'Arkime, Kibana'

  * **Notes:** 'All methods to perform Kerberoasting require sending a TGS-REQ packet to the KDC and requesting the ticket. Monitoring for specific requests that have RC4 encryption enabled or for a specific IPs requesting services not normally used by that machine.'

  * **Query:** ```Input your query here```

